In [3]:
import os
import re
from datetime import datetime, timedelta
import yt_dlp
import requests
from bs4 import BeautifulSoup
import time

def validate_soundcloud_url(url):
    """Validate if the provided URL is a SoundCloud URL."""
    pattern = r'^https?://(?:www\.)?soundcloud\.com/[\w-]+/[\w-]+'
    return bool(re.match(pattern, url))

def download_soundcloud_audio(url, output_dir="downloads"):
    """Download audio from SoundCloud URL."""
    if not validate_soundcloud_url(url):
        print(f"Invalid SoundCloud URL: {url}")
        return False

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Configure yt-dlp options
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': os.path.join(output_dir, '%(title)s.%(ext)s'),
        'noplaylist': True,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'quiet': False,
        'verbose': False
    }

    # Download the audio
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            filename = ydl.prepare_filename(info)
            base, _ = os.path.splitext(filename)
            mp3_file = f"{base}.mp3"
            print(f"Downloaded: {mp3_file}")
            return mp3_file
    except Exception as e:
        print(f"Error downloading {url}: {str(e)}")
        return False

def get_soundcloud_urls_by_date_range(profile_url, start_date, end_date):
    """
    Get SoundCloud URLs within a specific date range.

    Args:
        profile_url: The SoundCloud profile URL (e.g., "https://soundcloud.com/radio-ergo")
        start_date: Start date (datetime object)
        end_date: End date (datetime object)

    Returns:
        A list of SoundCloud URLs within the specified date range
    """
    if not profile_url.endswith('/'):
        profile_url = profile_url + '/'

    # Fetch the profile page
    try:
        response = requests.get(profile_url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()
    except Exception as e:
        print(f"Error fetching profile page: {str(e)}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all track links
    track_urls = []
    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.startswith('/') and profile_url.split('/')[3] in href and '/sets/' not in href:
            full_url = f"https://soundcloud.com{href}"
            if validate_soundcloud_url(full_url):
                track_urls.append(full_url)

    # Filter URLs by date
    urls_in_range = []

    # Month names mapping (both full and abbreviated)
    month_names = {
        # Full month names
        'january': 1, 'february': 2, 'march': 3, 'april': 4, 'may': 5, 'june': 6,
        'july': 7, 'august': 8, 'september': 9, 'october': 10, 'november': 11, 'december': 12,
        # Abbreviated month names
        'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'jun': 6, 'jul': 7,
        'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
    }

    for url in track_urls:
        # Try to extract date from URL using regex for both formats
        # Format 1: DD-month-YYYY (e.g., 11-march-2025)
        # Format 2: DD-mon-YYYY (e.g., 09-mar-2025)
        date_match = re.search(r'(\d{1,2})-([a-z]+)-(\d{4})', url.split('/')[-1], re.IGNORECASE)

        if date_match:
            day, month_str, year = date_match.groups()

            # Convert month name to number
            month = month_names.get(month_str.lower())

            if month:
                try:
                    track_date = datetime(int(year), month, int(day))
                    if start_date <= track_date <= end_date:
                        urls_in_range.append(url)
                        print(f"Found matching URL: {url}")
                except ValueError:
                    # Skip invalid dates
                    continue
            else:
                print(f"Could not parse month: {month_str} in URL: {url}")
        else:
            # Print URLs that don't match the pattern for debugging
            print(f"URL does not match date pattern: {url}")

    return urls_in_range

def download_by_date_range(profile_url, start_date_str, end_date_str, output_dir=None, save_to_drive=True):
    """
    Download SoundCloud tracks within a date range.

    Args:
        profile_url: The SoundCloud profile URL
        start_date_str: Start date in format 'YYYY-MM-DD'
        end_date_str: End date in format 'YYYY-MM-DD'
        output_dir: Output directory (default: None, which uses date range as folder name)
        save_to_drive: Whether to save files to Google Drive (default: True)
    """
    # Parse dates
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

    # Create output directory name based on date range if not provided
    if not output_dir:
        output_dir = f"soundcloud_{start_date_str}_to_{end_date_str}"

    print(f"Searching for tracks from {start_date_str} to {end_date_str}...")

    # Option 1: Try to find using the profile page
    urls = get_soundcloud_urls_by_date_range(profile_url, start_date, end_date)

    # Option 2: If specific URLs are known to exist but weren't found, add them manually
    manual_urls = []

    # Check if a specific URL was requested but not found
    check_url = "https://soundcloud.com/radio-ergo/idaacadda-09-mar-2025"
    if check_url not in urls:
        current_date = start_date
        while current_date <= end_date:
            day = current_date.day
            month = current_date.strftime('%b').lower()  # Get abbreviated month
            year = current_date.year

            # Try different date formats
            potential_urls = [
                f"https://soundcloud.com/radio-ergo/idaacadda-{day:02d}-{month}-{year}",
                f"https://soundcloud.com/radio-ergo/idaacadda-{day}-{month}-{year}"
            ]

            for url in potential_urls:
                try:
                    response = requests.head(url)
                    if response.status_code == 200:
                        manual_urls.append(url)
                        print(f"Manually found URL: {url}")
                except:
                    pass

            current_date += timedelta(days=1)

    # Combine both lists and remove duplicates
    all_urls = list(set(urls + manual_urls))

    if not all_urls:
        print("No tracks found in the specified date range.")
        return []

    print(f"Found {len(all_urls)} tracks in the specified date range.")
    print("URLs found:")
    for url in all_urls:
        print(f"- {url}")

    print(f"\nStarting download of {len(all_urls)} SoundCloud audios to: {output_dir}")
    downloaded_files = []
    for url in all_urls:
        result = download_soundcloud_audio(url, output_dir)
        if result:
            downloaded_files.append(result)
        # Add a small delay to avoid overloading the server
        time.sleep(1)

    print("\nSummary:")
    print(f"Total URLs processed: {len(all_urls)}")
    print(f"Successfully downloaded: {len(downloaded_files)}")
    if downloaded_files:
        print("\nDownloaded files:")
        for file in downloaded_files:
            print(f"- {file}")

    # Save to Google Drive
    if save_to_drive and downloaded_files:
        drive_path = "/content/drive/MyDrive/" + output_dir
        print(f"\nSaving files to Google Drive at: {drive_path}")

        # Create directory in Drive if it doesn't exist
        if not os.path.exists(drive_path):
            os.makedirs(drive_path)

        # Copy files to Drive
        os.system(f"cp -r {output_dir}/* {drive_path}/")
        print(f"Files successfully saved to Google Drive")

    return downloaded_files

# You can also directly specify URLs if the automatic search doesn't work
def download_specific_urls(urls, output_dir=None, save_to_drive=True):
    """
    Download specific SoundCloud URLs.

    Args:
        urls: List of SoundCloud URLs to download
        output_dir: Output directory (default: None, which uses today's date)
        save_to_drive: Whether to save files to Google Drive (default: True)
    """
    if not output_dir:
        output_dir = f"soundcloud_radio_ergo__downloads_{datetime.now().strftime('%Y-%m-%d')}"

    print(f"Starting download of {len(urls)} specific SoundCloud URLs to: {output_dir}")
    downloaded_files = []
    for url in urls:
        result = download_soundcloud_audio(url, output_dir)
        if result:
            downloaded_files.append(result)
        # Add a small delay to avoid overloading the server
        time.sleep(1)

    print("\nSummary:")
    print(f"Total URLs processed: {len(urls)}")
    print(f"Successfully downloaded: {len(downloaded_files)}")
    if downloaded_files:
        print("\nDownloaded files:")
        for file in downloaded_files:
            print(f"- {file}")

    # Save to Google Drive
    if save_to_drive and downloaded_files:
        try:
            from google.colab import drive
            drive.mount('/content/drive')
            print("Google Drive mounted successfully")
        except:
            print("Not running in Google Colab or Drive already mounted")
        drive_path = "/content/drive/MyDrive/" + output_dir
        print(f"\nSaving files to Google Drive at: {drive_path}")

        # Create directory in Drive if it doesn't exist
        if not os.path.exists(drive_path):
            os.makedirs(drive_path)

        # Copy files to Drive
        os.system(f"cp -r {output_dir}/* {drive_path}/")
        print(f"Files successfully saved to Google Drive")

    return downloaded_files

In [2]:
profile_url = "https://soundcloud.com/radio-ergo"  # SoundCloud profile/channel URL
start_date = "2022-08-01"  # Format: YYYY-MM-DD
end_date = "2022-12-31"    # Format: YYYY-MM-DD
save_to_drive = False       # Set to False if you don't want to save to Google Drive

downloaded_files = download_by_date_range(profile_url, start_date, end_date, save_to_drive=save_to_drive)

Searching for tracks from 2022-08-01 to 2022-12-31...


URL does not match date pattern: https://soundcloud.com/radio-ergo/likes
URL does not match date pattern: https://soundcloud.com/radio-ergo/sets
URL does not match date pattern: https://soundcloud.com/radio-ergo/tracks
URL does not match date pattern: https://soundcloud.com/radio-ergo/comments
Manually found URL: https://soundcloud.com/radio-ergo/idaacadda-01-aug-2022
Manually found URL: https://soundcloud.com/radio-ergo/idaacadda-1-aug-2022
Manually found URL: https://soundcloud.com/radio-ergo/idaacadda-02-aug-2022
Manually found URL: https://soundcloud.com/radio-ergo/idaacadda-2-aug-2022
Manually found URL: https://soundcloud.com/radio-ergo/idaacadda-03-aug-2022
Manually found URL: https://soundcloud.com/radio-ergo/idaacadda-3-aug-2022
Manually found URL: https://soundcloud.com/radio-ergo/idaacadda-04-aug-2022
Manually found URL: https://soundcloud.com/radio-ergo/idaacadda-4-aug-2022
Manually found URL: https://soundcloud.com/radio-ergo/idaacadda-05-aug-2022
Manually found URL: https

ERROR: [soundcloud] Unable to download JSON metadata: HTTP Error 404: Not Found (caused by <HTTPError 404: Not Found>)


Error downloading https://soundcloud.com/radio-ergo/idaacadda-10-nov-2022: ERROR: [soundcloud] Unable to download JSON metadata: HTTP Error 404: Not Found (caused by <HTTPError 404: Not Found>)
[soundcloud] Extracting URL: https://soundcloud.com/radio-ergo/idaacadda-17-nov-2022
[soundcloud] radio-ergo/idaacadda-17-nov-2022: Downloading info JSON
[soundcloud] 1384916305: Downloading hls_mp3 format info JSON
[soundcloud] 1384916305: Downloading http_mp3 format info JSON
[soundcloud] 1384916305: Downloading hls_opus format info JSON
[info] 1384916305: Downloading 1 format(s): hls_opus_0_0
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 358
[download] Destination: soundcloud_2022-08-01_to_2022-12-31/IDAACADDA 17-NOV-2022.opus
[download] 100% of   28.28MiB in 00:00:06 at 4.17MiB/s                    


ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location


...

ERROR: [soundcloud] Unable to download JSON metadata: HTTP Error 404: Not Found (caused by <HTTPError 404: Not Found>)


Error downloading https://soundcloud.com/radio-ergo/idaacadda-21-dec-2022: ERROR: [soundcloud] Unable to download JSON metadata: HTTP Error 404: Not Found (caused by <HTTPError 404: Not Found>)
[soundcloud] Extracting URL: https://soundcloud.com/radio-ergo/idaacadda-4-sep-2022
[soundcloud] radio-ergo/idaacadda-4-sep-2022: Downloading info JSON


ERROR: [soundcloud] Unable to download JSON metadata: HTTP Error 404: Not Found (caused by <HTTPError 404: Not Found>)


Error downloading https://soundcloud.com/radio-ergo/idaacadda-4-sep-2022: ERROR: [soundcloud] Unable to download JSON metadata: HTTP Error 404: Not Found (caused by <HTTPError 404: Not Found>)
[soundcloud] Extracting URL: https://soundcloud.com/radio-ergo/idaacadda-12-aug-2022
[soundcloud] radio-ergo/idaacadda-12-aug-2022: Downloading info JSON
[soundcloud] 1322855806: Downloading hls_mp3 format info JSON
[soundcloud] 1322855806: Downloading http_mp3 format info JSON
[soundcloud] 1322855806: Downloading hls_opus format info JSON
[info] 1322855806: Downloading 1 format(s): hls_opus_0_0
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 361
[download] Destination: soundcloud_2022-08-01_to_2022-12-31/IDAACADDA 12-AUG-2022.opus
[download]   0.4% of ~   7.76MiB at      0.00B/s ETA Unknown (frag 1/361)

KeyboardInterrupt: 